In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Configuration for using TPU with Fastai (Based on Pytorch)
###### faster,saves time as compared to GPU(as assignment deadline is short so instead of gpu using tpu) and support large memory so can use greater image target size)

In [ ]:
%%capture
!pip install -Uqq fastcore --upgrade
!pip install -Uqq fastai --upgrade
!pip install -Uqq cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl
!pip install -Uqq git+https://github.com/butchland/fastai_xla_extensions.git

In [ ]:
%%capture
import fastai_xla_extensions.core

In [ ]:
from fastai.vision.data import ImageDataLoaders
from fastai.vision.all import *
from fastai.imports import *
import gc 
import pandas as pd
import numpy as np
%matplotlib inline

# Dataset Preparation

In [ ]:
PATH = "../input/state-farm-distracted-driver-detection/"
TRAINING_PATH = "driver_imgs_list.csv"
CUSTOM_DATASET_PATH = "../input/state-farm-distracted-driver-detection/sample_submission.csv"

In [ ]:
train_df = pd.read_csv(PATH+TRAINING_PATH)
train_df["img"]="input/state-farm-distracted-driver-detection/imgs/train/"+train_df["classname"]+"/"+train_df["img"]

In [ ]:
train_df.head()

In [ ]:
train_df.tail()

In [ ]:
train_df=train_df.drop(columns=['subject'])
train_df=train_df[['img','classname']]
train_df.head()

# Data Loaders for fast ai setting image size to 450 X 450

In [ ]:
data = ImageDataLoaders.from_df(train_df,path="../",
                                 item_tfms=Resize(450),
                                 valid_pct=0.1,
                                 splitter=RandomSplitter(seed=42))
#for gpu cudnn
#seed=42
# , device=torch.device('cuda'))
# num_workers=0, device=torch.device('cuda')


# Dataset Sample

In [ ]:
data.show_batch(max_n=18, nrows=3)


# Downloading Pretrained Models -: resnet50 (pretrained on imagenet)   

In [ ]:
learn = cnn_learner(data, resnet50, metrics=accuracy, pretrained=True)

# Moving Model & Data to TPU & Finetuning

In [ ]:
learn.to_xla()
learn.fine_tune(2)

In [ ]:
learn.recorder.plot_loss()

# Result Analysis

## Confussion Metric

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(15,15), dpi=60)

## Most Confused Classes

In [ ]:
interp.most_confused(min_val=1)

In [ ]:
interp.plot_top_losses(6, nrows=2)

## Finding Optimal Learning Rate

In [ ]:
learn.lr_find()

In [ ]:
learn.unfreeze()

In [ ]:
learn.fit_one_cycle(2, lr_max=slice(1.2e-7, 1.5e-7))

# Loading Test Dataset

In [ ]:
test_dl = data.test_dl(get_image_files('../input/state-farm-distracted-driver-detection/imgs/test'),ordered=True)

In [ ]:
import os 
k=os.listdir("../input/state-farm-distracted-driver-detection/imgs/test")

# Predicting Results

In [ ]:
preds=learn.get_preds(dl=test_dl)

## converting Tensors to numpy array

In [ ]:
l=[]
for i in preds[0]:
    l.append(i.numpy())

In [ ]:
ans=pd.DataFrame(l)

In [ ]:
ans['img']=k

# Creating CSV of required format

In [ ]:
ans=ans[['img',0,1,2,3,4,5,6,7,8,9]]

In [ ]:
ans

In [ ]:
ans=ans.rename(columns={0:'c0',1:'c1',2:'c2',3:'c3',4:'c4',5:'c5',6:'c6',7:'c7',8:'c8',9:'c9'})

In [ ]:
ans.head()

In [ ]:
ans.to_csv("submission.csv",index=False)